In [4]:
import sqlite3 as sql
import pandas as pd
import glob as glob
import os
import zipfile

In [5]:
# dest_folder = '\\Databases'

In [6]:
db_name = "dvdrental.db"

In [62]:
origin = "csv_zip.zip"

In [63]:
#with sql.connect(os.path.join(dest_folder, db_name)) as conn:
with sql.connect(db_name) as conn:
    # df_dict = {}
    with zipfile.ZipFile(origin) as z:
        for file_name in z.namelist():
            if file_name.endswith('.csv'):
                table = os.path.splitext(file_name)[0]
                with z.open(file_name) as f:
                    df = pd.read_csv(f)
                    df.to_sql(table, conn, if_exists='replace', index=False)
                    # df_dict[os.path.splitext(file_name)[0]] = pd.read_csv(f)
                    print(f'Loaded {file_name} into table {table}')

Loaded country.csv into table country
Loaded film_category.csv into table film_category
Loaded store.csv into table store
Loaded staff.csv into table staff
Loaded film.csv into table film
Loaded city.csv into table city
Loaded address.csv into table address
Loaded customer.csv into table customer
Loaded inventory.csv into table inventory
Loaded category.csv into table category
Loaded actor.csv into table actor
Loaded language.csv into table language
Loaded film_actor.csv into table film_actor
Loaded rental.csv into table rental


In [66]:
# db = os.path.join(dest_folder, db_name)

In [18]:
with sql.connect(db_name) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
    tables = cursor.fetchall()

    print("Tables in db:")

    for table in tables:
        print("-", table[0])

Tables in db:
- country
- film_category
- store
- staff
- film
- city
- address
- customer
- inventory
- category
- actor
- language
- film_actor
- rental


In [61]:
def query_func(dbase, query:"str", params=None):
    try:
        with sql.connect(dbase) as conn:
            cursor = conn.cursor()
            cursor.execute(query, params or ())

            query_type = query.strip().split()[0].upper()
            print(f"Running a {query_type} query...")
    
            # Define behavior for SELECT queries
            if query.strip().upper().startswith("SELECT"):
    
                # Get column names
                columns = [desc[0] for desc in cursor.description]
        
                # Fetch data
                data = cursor.fetchall()
        
                # Return DataFrame        
                return pd.DataFrame(data, columns=columns)
            else:
                conn.commit()
                return cursor.rowcount
 
    except sql.OperationalError as e:
        print(f"Operational Error: {e}")
    except sql.DatabaseError as e:
        print(f"Database Error: {e}")
    except Exception as e:
        print(f"Unexpected Error: {e}")

In [62]:
df = query_func(db_name, 
            "SELECT * FROM country LIMIT 5;"
           
          )

df.head()

Running a SELECT query...


,country_id,country,last_update
0,1,Afghanistan,2006-02-15 09:44:00
1,2,Algeria,2006-02-15 09:44:00
2,3,American Samoa,2006-02-15 09:44:00
3,4,Angola,2006-02-15 09:44:00
4,5,Anguilla,2006-02-15 09:44:00


In [33]:
tables_clean = []
for table in tables:
    tables_clean.append(table[0])

tables_clean

['country',
 'film_category',
 'store',
 'staff',
 'film',
 'city',
 'address',
 'customer',
 'inventory',
 'category',
 'actor',
 'language',
 'film_actor',
 'rental']

In [63]:
df_dict = {}

for table in tables_clean:
    df_dict[table] = query_func(db_name, f"SELECT * FROM {table} LIMIT 5;")
    print(f"{table} was successfully added to df_dict")

Running a SELECT query...
country was successfully added to df_dict
Running a SELECT query...
film_category was successfully added to df_dict
Running a SELECT query...
store was successfully added to df_dict
Running a SELECT query...
staff was successfully added to df_dict
Running a SELECT query...
film was successfully added to df_dict
Running a SELECT query...
city was successfully added to df_dict
Running a SELECT query...
address was successfully added to df_dict
Running a SELECT query...
customer was successfully added to df_dict
Running a SELECT query...
inventory was successfully added to df_dict
Running a SELECT query...
category was successfully added to df_dict
Running a SELECT query...
actor was successfully added to df_dict
Running a SELECT query...
language was successfully added to df_dict
Running a SELECT query...
film_actor was successfully added to df_dict
Running a SELECT query...
rental was successfully added to df_dict


In [37]:
df_dict.keys()

dict_keys(['country', 'film_category', 'store', 'staff', 'film', 'city', 'address', 'customer', 'inventory', 'category', 'actor', 'language', 'film_actor', 'rental'])

In [46]:
for key in df_dict:
    print(key, ":\n\n", df_dict[key].head(), "\n\n")

country :

    country_id         country          last_update
0           1     Afghanistan  2006-02-15 09:44:00
1           2         Algeria  2006-02-15 09:44:00
2           3  American Samoa  2006-02-15 09:44:00
3           4          Angola  2006-02-15 09:44:00
4           5        Anguilla  2006-02-15 09:44:00 


film_category :

    film_id  category_id          last_update
0        1            6  2006-02-15 10:07:09
1        2           11  2006-02-15 10:07:09
2        3            6  2006-02-15 10:07:09
3        4           11  2006-02-15 10:07:09
4        5            8  2006-02-15 10:07:09 


store :

    store_id  manager_staff_id  address_id          last_update
0         1                 1           1  2006-02-15 09:57:12
1         2                 2           2  2006-02-15 09:57:12 


staff :

    staff_id first_name last_name  address_id                         email  \
0         1       Mike   Hillyer           3  Mike.Hillyer@sakilastaff.com   
1         2        J

In [57]:
df_dict["store"].head()

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2006-02-15 09:57:12
1,2,2,2,2006-02-15 09:57:12
